In [1]:
from PIL import Image
import os, glob
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

2023-03-28 17:08:50.284718: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [53]:
dijet = np.load("jj_nbm.npy",allow_pickle=True)
tttt = np.load("tttt_nbm.npy",allow_pickle=True)

In [54]:
dijet_lab = []
tttt_lab = []

for ii in range(len(dijet)):
    dijet_lab.append([0,1])

for ii in range(len(tttt)):
    tttt_lab.append([1,0])

    
dijet_label = np.array(dijet_lab)
tttt_label = np.array(tttt_lab)

In [55]:
X = np.concatenate((dijet,tttt),axis=0)
X

array([[array([[0., 0., 0., ..., 0., 0., 0.],
               [0., 0., 0., ..., 0., 0., 0.],
               [0., 0., 0., ..., 0., 0., 0.],
               ...,
               [0., 0., 0., ..., 0., 0., 0.],
               [0., 0., 0., ..., 0., 0., 0.],
               [0., 0., 0., ..., 0., 0., 0.]]), 0],
       [array([[0., 0., 0., ..., 0., 0., 0.],
               [0., 0., 0., ..., 0., 0., 0.],
               [0., 0., 0., ..., 0., 0., 0.],
               ...,
               [0., 0., 0., ..., 0., 0., 0.],
               [0., 0., 0., ..., 0., 0., 0.],
               [0., 0., 0., ..., 0., 0., 0.]]), 0],
       [array([[0., 0., 0., ..., 0., 0., 0.],
               [0., 0., 0., ..., 0., 0., 0.],
               [0., 0., 0., ..., 0., 0., 0.],
               ...,
               [0., 0., 0., ..., 0., 0., 0.],
               [0., 0., 0., ..., 0., 0., 0.],
               [0., 0., 0., ..., 0., 0., 0.]]), 0],
       ...,
       [array([[0., 0., 0., ..., 0., 0., 0.],
               [0., 0., 0., ..., 0.,

In [58]:
Y = np.concatenate((dijet_label,tttt_label),axis=0)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [60]:
categories = ["tttt","dijet"]
nb_classes = len(categories)

In [61]:
image_w = 64 
image_h = 64

In [66]:
X_train.reshape(150000,64,64,1)

ValueError: cannot reshape array of size 300000 into shape (150000,64,64,1)

In [63]:
X_train = X_train[:][:].astype("float") / 256
X_test  = X_test[:][:].astype("float")  / 256

ValueError: setting an array element with a sequence.

In [64]:
X_train_ts = X_train.reshape(150000,64, 64, 1, 1)

ValueError: cannot reshape array of size 300000 into shape (150000,64,64,1,1)

In [ ]:
X_train_ts.shape

In [ ]:
y_train.shape

In [ ]:
y_train


In [ ]:
X_train_ts

In [ ]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, 
    input_shape=(64,64,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='valid'))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(1, 1), padding='valid'))
model.add(Dropout(0.25))

model.add(Flatten()) # --- (※3) 
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy'])



In [ ]:
model.fit(X_train_ts, y_train, batch_size=32, epochs=50, workers=3)

In [ ]:
X_test_ts = X_test.reshape(50000,64, 64, 1, 1)

In [ ]:
y_pre = model.predict(X_test_ts,workers=3)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(y_pre[:,0][y_test[:,1] == 1], bins=[0.,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
plt.hist(y_pre[:,0][y_test[:,1] == 0], bins=[0.,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0], histtype='step')
plt.title("prediction")
plt.show()